In [1]:
import json


def parse_utterance_block(block_lines):
    """解析单个话语块的数据"""
    # 解析第一行
    first_line = block_lines[0].strip().split('\t')
    time_str = first_line[0].strip('[]').split(' - ')

    utterance = {
        "timing": {
            "start": float(time_str[0]),
            "end": float(time_str[1])
        },
        "id": first_line[1],
        "final_emotion": first_line[2],
        "dimensional_average": {
            "valence": float(first_line[3].strip('[]').split(', ')[0]),
            "activation": float(first_line[3].strip('[]').split(', ')[1]),
            "dominance": float(first_line[3].strip('[]').split(', ')[2])
        }
    }

    return utterance


def create_json_structure(text, session_id):
    """创建完整的JSON结构"""
    lines = text.split('\n')
    utterances = []
    current_block = []

    for line in lines:
        if line.strip() == "" or line.startswith('%'):
            if current_block:
                utterances.append(parse_utterance_block(current_block))
                current_block = []
            continue
        current_block.append(line)

    if current_block:
        utterances.append(parse_utterance_block(current_block))

    return {
        "session_info": {
            "session_id": session_id,
        },
        "utterances": utterances
    }

In [3]:
import os

for i in range(5):
    path = f'/mnt/users/whaledolphin/FER-tts/MIKU-PAL-test/IEMOCAP_full_release/Session{i+1}/dialog/EmoEvaluation'
    files = os.listdir(path)

    txt_files = [f for f in files if f.endswith('.txt')]

    for txt_file in txt_files:
        with open(os.path.join(path, txt_file), 'r') as f:
            text = f.read()
        json_data = create_json_structure(text, txt_file.split('.')[0])
        with open(f'/mnt/users/whaledolphin/FER-tts/MIKU-PAL-test/IEMOCAP_full_release/IEMOCAP_session{i+1}.json', 'a') as json_file:
            json.dump(json_data, json_file, ensure_ascii=False, indent=4)

In [1]:
import sys
sys.path.append('models/dsfd')

In [ ]:
from models.dsfd.inference import FaceDetect

fd = FaceDetect(
    confidence_threshold=0.5,
    nms_iou_threshold=0.3,
    max_resolution=640,
    device='cuda:0',
    fp16_inference=True,
    clip_boxes=True,
    model_path='checkpoints/dsfd.pth',
    batch_size=64,
)